<center><img src="Slide1.png" width = "800"/></center>

### <b> InSAR Coherent Change Detection over Aleppo, Syria (2016)
    
<b><i> Corey Scher<sup>1</sup>
<i>Jamon van Den Hoek<sup>2</sup>
    
<i><font size=2> 1. Department of Earth and Environmental Sciences | Graduate Center at the City University of New York
    
<i><font size=2> 2. College of Earth, Ocean, and Atmospheric Sciences | Oregon State University

### Introduction

   In this training we will walk through an example of bi-temporal coherent change detection over Aleppo, Syria to map proxies for urban damage that occurred as a result of air bombardments in 2016. We will be using the [Alaska Satellite Facility's](https://asf.alaska.edu/) [Vertex tool](https://search.asf.alaska.edu/#/) to generate two synthetic aperture radar [interferogram](https://appliedsciences.nasa.gov/sites/default/files/Session4-SAR-English_0.pdf) (InSAR) datasets to look for anomalous decreases in InSAR measurements of [coherence](https://en.wikipedia.org/wiki/Coherence_(physics) that occurred following a series of airstrike bombardments. This workflow can be completed either using the ASF [Hyp3 Python SDK](https://hyp3-docs.asf.alaska.edu/), as we will demonstrate in this notebook, or generation of InSAR coherence images can be accomplished manually using the ASF Vertex [user interface](https://search.asf.alaska.edu/#/) and raster band math for detection of anomalous areas of changes in InSAR coherence using a geographic information system interface, such as [QGIS](https://www.qgis.org/en/site/).